### Automation - Order Processing
- 1. Gmail API 연동 후 Order or Cancel 메일 가져오기
- 2. Order or Cancel 데이터 전처리
- 3. 암호화, API를 통해 Database에 전송


## 메일 확인

In [2]:
import datetime as dt, pandas as pd, datetime as dt
import os, time, pymysql, re, requests, pickle, os.path, base64, email, hashlib, crypto, sys
from openpyxl import load_workbook
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient.errors import HttpError
from base64 import b64encode
sys.modules['Crypto'] = crypto
from Crypto.Util.Padding import pad
from Crypto.Cipher import AES

def ts(num : int):
    time.sleep(num)

def live_db_conn():
    conn = pymysql.connect(host='host', user='user', password='password',autocommit=True,cursorclass=pymysql.cursors.DictCursor, db = "db")
    return conn

def data_encoder(text):
    if len(text)>0:
        message = email.message_from_string(str(base64.urlsafe_b64decode(text), 'utf-8'))
    return message

def change_time(x):
    x = x.lower().replace(".",'').replace(" ",'').replace("::",':').replace("點",'')
    if 'pm' in x.lower() :
        if len(re.sub(r'[^0-9]','',x)) == 3 :
            x = '{}:{}'.format(12+int(x[0]),x[1:]).replace("::",':')
        elif len(x.lower().replace("pm",'')) == 1 :
            x = "{}:00".format(int(x.lower().replace("pm",'')) + 12)
    elif len(x)== len(re.sub(r'[^0-9]','',x)) == 4 :
        x = "{}:{}".format(x[:2],x[2:]) 
    elif len(re.sub(r'[^0-9]','',x)) == 3 :
        x = "{}{}".format(int(x[0])+12, x[1:]) 
    return x

def get_date(string):
    y, m = string.strip()[12:].split()[0], dt.datetime.strptime(string.strip()[8:].split()[0],'%b').month#.tm_mon
    d,t = string.strip()[5:].split()[0]  , string.strip()[17:25]
    bt = "{}:{}:{} {}".format(y,m,d,t)
    result = dt.datetime.strptime(bt , "%Y:%m:%d %H:%M:%S") + dt.timedelta(hours=16)
    return result.strftime("%Y-%m-%d %H:%M:%S")

def get_cancel_order_no(ota_order_num):

    conn = live_db_conn()
    curs = conn.cursor()


    sql = """
    SELECT *
    FROM `order` o
    WHERE o.status != 'cancel' AND o.ota_order_no = '{}'
    """.format(ota_order_num)
    curs.execute(sql)
    cancel_order = pd.DataFrame(curs.fetchall())

    curs.close()
    conn.close()

    canel_order_no = cancel_order['order_no'].values[0]

    return canel_order_no

def get_ota_store():

    conn = live_db_conn()
    curs = conn.cursor()

    # Klook 계약 매장 
    sql = """SELECT OS.store_id, S.name ,OS.store_id_ext, OS.memo, SC.order_confirm
    FROM ota_store OS, store S ,store_contract SC 
    WHERE OS.ota_id = 3 AND OS.store_id = S.id AND SC.store_id = S.id"""
    curs.execute(sql)
    df = pd.DataFrame(curs.fetchall())

    curs.close()
    conn.close()

    return df

def get_ota_product():

    conn = live_db_conn()
    curs = conn.cursor()

    # Klook 계약 매장의 상품
    sql = """SELECT OP.memo, OP.product_id, OP.product_id_ext, price FROM ota_product OP WHERE OP.ota_id = 3"""
    curs.execute(sql)
    df = pd.DataFrame(curs.fetchall())

    curs.close()
    conn.close()

    return df

def get_order():

    conn = live_db_conn()
    curs = conn.cursor()

    # 예약 건 확인
    sql = """SELECT O.store_id, O.admin_id, O.manager_id , O.order_at , case when O.confirm_at is null then 0 else 1 end 'confirm_at', O.ota_order_no , O.manager_memo, O.name, O.status, SC.order_confirm, O.need_confirm
    FROM `order` O
    LEFT JOIN store_contract SC ON SC.store_id = O.store_id
    WHERE O.ota_id = 3"""
    curs.execute(sql)
    df = pd.DataFrame(curs.fetchall())

    curs.close()
    conn.close()

    return df

def encryptString(str, secret_key, secret_iv):
    key = hashlib.sha256(secret_key.encode('utf-8')).hexdigest()[:32].encode('utf-8')
    iv = hashlib.sha256(secret_iv.encode('utf-8')).hexdigest()[:16].encode('utf-8')
    data = pad(str.encode(), AES.block_size)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    return b64encode(b64encode(cipher.encrypt(data)).decode("utf-8").encode()).decode("utf-8").replace("=","")

In [2]:
def KLOOK(N_days):
    
    start_day = time.localtime().tm_mday
    
    ota_store = get_ota_store()
    ota_product = get_ota_product()
    order = get_order()

    # Google Mail API 사용 토큰, Google Mail API 사용 용도, REDTABLE 예약 및 취소 API 
    creds, SCOPES, URL_LIVE_CREATE, URL_LIVE_CANCEL  = None, ['https://www.googleapis.com/auth/gmail.readonly'], 'CREATE_URL', 'CANCEL_URL'
    
    # REDTABLE 암호화 정보
    secret_key, secret_iv = "secret_key", "secret_iv"

    # 특별 케이스 Dictionary 생성: 더플레이스 다이닝, 엔그릴, 한쿡
    SIE_EXCEPT = {'94598':'99979', '94439':'99975', '93844':'99921'} 
    PN_EXCEPT = {'Special Namsan Course Menu(For 1/Weekday Dinner/17:00~22:00)': 'N Seoul Tower Ticket & Namsan Course Menu(Weekday Dinner/17:00~22:00)',
                    'Special Namsan Course Menu(For 1/Weekend and Holiday)': 'N Seoul Tower Ticket & Namsan Course Menu(Weekend and Holiday)'}
    
    # 불필요한 단어 리스트 (후에 제거)
    re_word = "Booking reference ID:|Date Request:|Time Request:|Lead participant:|Country/region of passport:|Participant:|Activity URL:|Number of Diners:"

    while True:
        
        current_day = time.localtime().tm_mday
        day_diff = current_day - start_day
        current_hour = time.localtime().tm_hour

        # N_days일 후, 오전 5시 이후면 3시간 뒤면 8시 넘으니까 코드 중지. 출근해서 직접 돌리면 됨.
        if N_days > 0 and  day_diff == N_days and current_hour >= 5:
            print('3시간 뒤 8시를 넘으므로 코드를 중지합니다.\n현재 시간: ', dt.datetime.now())
            break
        
        # Token을 통해 Credentials 생성
        if os.path.exists('token.pickle'):

            with open('token.pickle', 'rb') as token:

                creds = pickle.load(token)

        # Token이 만료되거나 문제가 있는 경우        
        if not creds or not creds.valid:
            
            # Refresh
            if creds and creds.expired and creds.refresh_token:
                
                creds.refresh(Request())
                
            # 해당 경로를 통해 Credentials 생성
            else:

                flow = InstalledAppFlow.from_client_secrets_file('client_secrets_file', SCOPES)
                creds = flow.run_local_server(port=0)
            
            # 정상적인 token, credentials 생성
            with open('token.pickle', 'wb') as token:
                
                pickle.dump(creds, token)

        start = time.time()        

        # BUILD - Google Mail API  
        service = build('gmail', 'v1', credentials=creds)
    
        print('============================================')
        print('<Part 1. 취소 메일>')
        print('취소 건 확인을 시작합니다.')
        
        try:

            cancel_messages = service.users().messages().list(userId='me', q='subject:Klook Order Canceled -Reminder').execute()['messages']
            print('정상적으로 취소 메일을 불러왔습니다.')

        except Exception as load_error:
            
            print('취소 메일 불러오기 로딩 실패: ', load_error)
            # print('요청 횟수 초과로 예약취소 메일 접근이 제한되었습니다. 권장 시간 이후에 진행해주세요.')

        cancel_cnt = 0
        for message_id in cancel_messages:

            message = service.users().messages().get(userId='me', id= message_id['id']).execute()
            
            try : 
                # 취소 메일 속에서 주문 번호를 확인
                if 'order' in message['snippet'].lower() and 'cancel' in message['snippet'].lower() and 'amendment' not in message['snippet'].lower():
        
                    if 'Booking reference ID: ' in message['snippet']:

                       ota_order_no = message['snippet'].split("Booking reference ID: ")[1].split()[0].strip()

                    elif 'parts' in message['payload'] and 'Booking reference ID: ' in str(data_encoder(message['payload']['parts'][0]['body']['data'])):

                        ota_order_no = str(data_encoder(message['payload']['parts'][0]['body']['data'])).split('Booking reference ID: ')[1].split("\n")[0]

                    elif 'parts' not in message['payload']:

                        ota_order_no = pd.read_html(str(data_encoder(message['payload']['body']['data'])))[5][1][0].split("Booking reference ID: ")[1].strip().split()[0]

                    else:
                        
                        text = "취소된 주문번호를 찾을 수 없습니다."
                        print(text)
                    
                    # 해당 OTA_Order_No의 order_no를 찾아서 API로 전송
                    if len(order[(order['ota_order_no']== ota_order_no) & (order['status'] !='cancel') & (order['status'] != 'deny') ]) > 0:
                        
                        cancel_order_no = get_cancel_order_no(ota_order_no)

                        PARAMS = """
                        { 
                        'ota_code': 'klook',
                        'ota_key': 'L2w5SGIvUFcrbzZzc3huSnVZSjhnZz09',
                        'order_no': '%s'}
                        """ % cancel_order_no
                        
                        data = PARAMS.replace(',',',').replace("'",'"').replace(" && ","&&").replace('"[','[').replace(']"',']').replace("：",':').replace(" : ",':').replace("  ",' ').replace("Special requirements:",'').replace("{store",'{"store').replace('pm"','"').replace("::",":").replace('"d ','`d')
                    
                        r = encryptString(data, secret_key, secret_iv).replace("=",'')
                    
                        req = requests.post(url = URL_LIVE_CANCEL, data = {'data' : r})

                        if req.status_code == 200:

                            print('{} - 취소 반영 완료 '.format(ota_order_no))
                            cancel_cnt += 1
                            
                        else:

                            print(req, '{} - 취소 실패되었습니다.'.format(ota_order_no))
                            print(req.content)
                        
                    # 취소가 아니거나 거절인 경우
                    else: continue

                # 예약 내용 변경 취소인 경우에
                else: continue

            except Exception as cancel_error:
                print(cancel_error)
                text = "{} - 취소 갱신 에러: 확인필요".format(ota_order_no)
                print(text)

        print('취소 건 확인 완료. 취소 건은 총', cancel_cnt,'건입니다.')
        print('--------------------------------------------')

        print('<Part 2. 예약 메일>')
        print('예약 건 확인을 시작합니다.')
        try:
        
            messages = service.users().messages().list(userId='me', q='subject:Klook has received an order -Dear team').execute()['messages']
            print('정상적으로 예약 메일을 불러왔습니다.')

        except Exception as load_error:
            
            print(load_error)
            
        

        # 기본 데이터프레임 
        df = pd.DataFrame(columns= ['store_id', 'products', 'total_price', 'pay_method', 'currency','booking_time', 'reserve_day', 'reserve_time', 'name', 'email', 'phone','memo', 'person', 'ota_code', 'ota_key', 'ota_sns_type', 'ota_sns_id','ota_country', 'ota_order_no', 'confirm_at', 'order_confirm'])

        # 바우처 번호를 부야해주기 위한 틀 생성
        oo = order[(order['ota_order_no'].notnull()) & (order['manager_memo'].notnull())]['manager_memo'].apply(lambda x : str(x).split(":")[1].split(",")[0].strip().replace('요청사항',''))
        
        # 당일 예약 건이 이미 존재하면, 해당 건의 다음 번호를 부여
        if len(oo[oo.str[:6] == dt.datetime.now().strftime("%y%m%d")].apply(lambda x : int(x[-4:]))) > 0:    
            
            voucher_num = max(oo[oo.str[:6] == dt.datetime.now().strftime("%y%m%d")].apply(lambda x : int(x[-4:]))) + 1

        # 당일 첫 예약 건인 경우, 1 부여
        else: voucher_num = 1

        # 예약 내용 확인
        for message_id in messages:
            

            message = service.users().messages().get(userId='me', id= message_id['id']).execute()

            contents = pd.read_html(str(data_encoder(message['payload']['body']['data'])))[6][0]
            sie = contents[6].split("/")[-1]
            pn = message['snippet'][51:].split(" - ")[1].split("Please")[0][:-2].replace("amp;",'').strip()
            
            tmp = pd.DataFrame(map(lambda x : re.sub(re_word ,"", x).strip(), contents[:6].values)).transpose()
            tmp.columns = ['ota_order_no','reserve_day','reserve_time','name','ota_country','person']
            tmp['memo'] = 'X'

            ota_order_no = tmp['ota_order_no'][0]
            ota_order_no_list = order[order['ota_order_no'].notnull()]['ota_order_no'].unique().tolist()

            # 이미 등록된 예약 건은 통과
            if ota_order_no in ota_order_no_list: continue

            try: 
                for content in contents[5:]: #8
                    
                    if 'Preferred' in content:
                        
                        tmp['reserve_time'] = content.split('  ')[1]
                            
                    elif 'Number of Diners:' in content:

                        tmp['person'] = str(re.sub(re_word,'', content)).strip()
                    
                    elif 'No. of participants:' in content:
        
                        tmp['person'] = content.split(':')[1].strip()
                        
                    else:

                        tmp['memo'] = content

                tmp['booking_time'] = get_date(message['payload']['headers'][3]['value'].split(";")[1])[:-3]
                
                person_info = tmp['person'][0]
                info_list = person_info.split(', ')

                # 특별 케이스
                if pn in PN_EXCEPT.keys(): pn = PN_EXCEPT[pn]
                if sie in SIE_EXCEPT.keys(): sie = SIE_EXCEPT[sie]

                # STORE_ID 조회 (실패할 경우, 다음 예약 건으로 넘어감)
                if sie in dict(ota_store[['store_id_ext','store_id']].values):
                    
                    tmp['store_id'] = dict(ota_store[['store_id_ext','store_id']].values).get(sie)
                    store_id = tmp['store_id'][0]
                    tmp['order_confirm'] = dict(ota_store[['store_id_ext','order_confirm']].values).get(sie)
                    tmp['confirm_at'] = dict(order[['store_id','confirm_at']].values).get(store_id)

                else:

                    print('일치하는 STORE_ID_EXT {}가 존재하지 않습니다.')
                    continue
                    



                Product_Quantity = {}
            
            
                for info in info_list:

                    info_split = info.split(' x ')

                    quantity = info_split[0].strip()
                    memo_or_size = info_split[1].strip()
                    
                    # 단순 Person or People or Set 표기만 되어 있는 경우
                    
                    if ('Person' in memo_or_size) or ('Set' in memo_or_size) or ('people' in memo_or_size) or (pn in memo_or_size): pn_size = pn
                    
                    # 사이즈, 금액권 등 상품명 정보가 포함되어 있어, MEMO Input 값을 변경해줘야하는 경우
                    else: pn_size = pn + '-' + info_split[1].strip().replace(pn,'')

                    Product_Quantity[pn_size] = quantity
                    
                
                Order_Info_List = []

                for product_name, quantity in Product_Quantity.items():

                    if product_name in dict(ota_product[['memo','product_id']].values):
                        
                        Order_Info_List.append({"product_id" : str(dict(ota_product[['memo','product_id']].values).get(product_name)) , 'product_price' : int(dict(ota_product[['memo','price']].values).get(product_name)), 'product_quantity' : int(quantity)})

                        tmp['products'] = [Order_Info_List]
                        
                    else:
                                
                        text = 'DB에 해당 Product가 존재하지않습니다.: {}'.format(product_name)
                        print(text)
                    
                product_list = tmp['products'][0]
                tmp['total_price'] = sum([product['product_quantity'] * product['product_price'] for product in product_list])

                # MEMO
                yymmdd = dt.datetime.now().strftime("%y%m%d")
                origin_memo = tmp['memo'][0]
                voucher_num += 1
                if voucher_num < 10: etc_memo = "바우처 번호: {}-REDTABLE-000{} , 요청사항: {}".format(yymmdd, voucher_num, origin_memo)
                else: etc_memo = "바우처 번호: {}-REDTABLE-00{} , 요청사항: {}".format(yymmdd, voucher_num, origin_memo)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
                tmp['memo'] = etc_memo
            
                df = pd.concat([df,tmp])

            except:

                print("{}건에 대해 오류가 있어 다음 예약 건으로 넘어갑니다.")
            
        if len(df) != 0:

            df['reserve_time'] = df['reserve_time'].apply(lambda x : change_time(x))
            df['reserve_time'] = df['reserve_time'].apply(lambda x : x.replace('[sat-sun]',''))
            df['person'] = df['person'].apply(lambda x : x.replace(" x ",' ').replace("adults ","adults, "))
            df['name'] = df['name'].apply(lambda x : x.replace("()",'').replace(")",') '))
            df['products'] = df['products'].apply(lambda x : str([x]) if isinstance(x,dict) else x)
            
            df[['pay_method','currency','ota_code','ota_key']] = ['card','KRW', 'klook', 'L2w5SGIvUFcrbzZzc3huSnVZSjhnZz09']
            
            df[['store_id','total_price','person']] = df[['store_id','total_price','person']].astype(str)
            
            df['memo'] = df['memo'].apply(lambda x : x.replace("-0",'-') if len(x.split("-REDTABLE-")[1].split()[0]) == 5 else x).str.replace(u"\xa0",' ').replace('don"t',"don't")
            
            df['reserve_time'] = df['reserve_time'].apply(lambda x : x if x != 'NA' else '00:00')
            
            df['ota_order_no'] = df['ota_order_no'].str.strip()
            
            df.fillna(inplace = True, value='')

            df['reserve_time'] = df['reserve_time'].str.replace("am",'')
            df['reserve_time'] = df['reserve_time'].str.replace("[mon-fri]",'')
            print("DF 개수: {}, 실제 주문 처리 대상 개수: {}".format(len(df),len(df['ota_order_no'].unique())))
            df.drop_duplicates('ota_order_no',inplace=True,keep='first')

            reserve_cnt = 0
            for i in range(len(df)):
            
                ota_order_no = df.iloc[i]['ota_order_no']
                print('주문번호 {}을 DB에 등록합니다'. format(ota_order_no))

                PARAMS = str(df.iloc[i,:-2].to_dict())
                data = PARAMS.replace("'",'"').replace(" && ","&&").replace('"[','[').replace(']"',']').replace("：",':').replace(" : ",':').replace("  ",' ').replace("Special requirements:",'').replace("{store",'{"store').replace('pm"','"').replace("::",":").replace('"d ','`d')
                r = encryptString(data, secret_key, secret_iv).replace("=",'')

                req = requests.post(url = URL_LIVE_CREATE, data = {'data' : r})

                if req.status_code == 200:

                    print('{} - DB 등록 완료'.format(df.iloc[i]['ota_order_no']))
                    reserve_cnt += 1

                else: 

                    text = '{} - DB 등록 실패'.format(df.iloc[i]['ota_order_no'])
                    
                    print(text)
                    print(req.content)

            conn = live_db_conn()
            curs = conn.cursor()
            sql = """
            UPDATE `order` O
            SET O.manager_memo = SUBSTRING_INDEX(O.memo, ",",1), O.memo = REPLACE(O.memo, CONCAT(SUBSTRING_INDEX(O.memo, ",",1),", "), "")
            WHERE LEFT(O.memo , 7) = "바우처 번호:" AND O.manager_memo IS NULL
            """
            curs.execute(sql)
            curs.close()
            conn.close()

            print('모든 건이 예약이 완료되었습니다.')
            print('예약 건 확인 완료. 예약 건은 총', reserve_cnt,'건입니다.')
            print('--------------------------------------------')
            
        else:

            print('추가 등록된 예약 건이 없습니다.')

        print('--------------------------------------------')
        print("소요 시간 : {}".format(time.time() - start))
        print("완료 시간 : {}".format(dt.datetime.now()))
        print('============================================')


        if N_days == 0: break
        
        else:

            print('3시간 뒤 코드 재실행됩니다.')
            print('============================================')
            ts(3600*3)

    return df

In [3]:
df = KLOOK(0)

<Part 1. 취소 메일>
취소 건 확인을 시작합니다.
정상적으로 취소 메일을 불러왔습니다.
취소 건 확인 완료. 취소 건은 총 0 건입니다.
--------------------------------------------
<Part 2. 예약 메일>
예약 건 확인을 시작합니다.
정상적으로 예약 메일을 불러왔습니다.
추가 등록된 예약 건이 없습니다.
--------------------------------------------
소요 시간 : 69.6085786819458
완료 시간 : 2024-01-10 15:26:29.304381
